In [ ]:
import torch
import numpy as np
from torch import nn
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.__version__

## 1. Data (preparing and loading)

Data can be almost anything... in machine learning

* Excel spreadsheet
* Images of any kind
* Videos
* Audio
* DNA
* Text, etc.

Machine learning is a game of two parts:
1. Get data into a numerical representation
2. Build a model to learn patterns in that numerical representation

To showcase this, let's create some *known* data using the linear regression formula. We'll use a linear regression formula to make a straight line with *known* **parameters**.

In [ ]:
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X = X.to(device)
y = y.to(device)

X[:10], y[:10]

In [ ]:
len(X), len(y)

### Splitting data into training and test sets

In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
# visualize our data
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
    """
    Plots training data, test data and compares predictions
    :param train_data:
    :param train_labels:
    :param test_data:
    :param test_labels:
    :param predictions:
    :return:
    """

    plt.figure(figsize=(10, 7))
    plt.scatter(train_data.cpu(), train_labels.cpu(), c="b", s=4, label="Training data")
    plt.scatter(test_data.cpu(), test_labels.cpu(), c="g", s=4, label="Testing data")

    if predictions is not None:
        plt.scatter(test_data.cpu(), predictions.cpu(), c="r", s=4, label="Predictions")

    plt.legend(prop={"size": 14})

In [ ]:
plot_predictions()

## 2. Build model (linear regression)

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad=True,
                                             dtype=torch.float))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.weights * x + self.bias

### Checking the contents of our model

In [ ]:
torch.manual_seed(42)

model_0 = LinearRegressionModel()

model_0.to(device)

list(model_0.parameters())

In [ ]:
model_0.state_dict()

### Making prediction using `torch.inference_mode()`

To check our model's predictive power, let's see how well it predicts `y_test` based on `X_test`.

When we pass data through our model, it will pass data through `forward()` method.

In [ ]:
with torch.inference_mode():
    y_preds = model_0(X_test)

y_preds

In [ ]:
plot_predictions(predictions=y_preds)

## 3. Train model

The whole idea of training is for a model to move from some *unknown* parameters to some *known* ones. It can be done using loss function.

**Loss function:** A function to measure how wrong your model's predictions are to the ideal outputs, lower is better.
**Optimized:** Takes into account the loss of a model and adjusts the model's parameters to improve lose function.

In [ ]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model_0.parameters(),
                            lr=0.01)

### Building a training loop in PyTorch
1. Look through data
2. Forward pass (**forward propagation**)
3. Calculate loss (compare predictions to ground truth)
4. Optimizer zero grad
5. Loss backward (calculate gradients for each parameter - **backpropagation**)
6. Optimizer step (adjust parameters based on loss)

In [ ]:
epochs = 200

epoch_count = []
loss_values = []
test_loss_values = []

for epoch in range(epochs):
    model_0.train()  # set model to training mode

    y_pred = model_0(X_train)  # forward pass
    loss = loss_fn(y_pred, y_train)  # calculate the loss
    optimizer.zero_grad()  # optimizer zero grad
    loss.backward()  # backpropagation
    optimizer.step()  # update parameters

    model_0.eval()

    with torch.inference_mode():
        test_pred = model_0(X_test)
        test_loss = loss_fn(test_pred, y_test)

    if epoch % 10 == 0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(test_loss)

In [ ]:
model_0.state_dict()

In [ ]:
with torch.inference_mode():
    test_pred = model_0(X_test)
    test_loss = loss_fn(test_pred, y_test)

In [ ]:
plot_predictions(predictions=test_pred)

In [ ]:
plt.plot(epoch_count, np.array(torch.tensor(loss_values).cpu().numpy()), label="Train Loss")
plt.plot(epoch_count, np.array(torch.tensor(test_loss_values).cpu().numpy()), label="Test Loss")
plt.title("Training and test loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

## Saving a model in PyTorch
1. `torch.save()` - allows to save a PyTorch object in Python's pickle format
2. `torch.load()` - allows to load a saved PyTorch object
3. `torch.nn.Module.load_state_dict()` - allows to load a model's saved state dictionary

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "linear_regression_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

torch.save(obj=model_0.state_dict(),
           f=MODEL_SAVE_PATH)

## Loading a PyTorch model

Since we saved our model's `state_dict()` rather than entire model, we'll create a new instance of our model class and load the saved `state_dict()` into that.

In [ ]:
loaded_model_0 = LinearRegressionModel()
loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_model_0.to(device)

In [ ]:
loaded_model_0.eval()
with torch.inference_mode():
    loaded_model_0_preds = loaded_model_0(X_test)

plot_predictions(predictions=loaded_model_0_preds)